In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sqlite3
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from config import settings

In [3]:
settings.sqlite_path = '/Users/mert/Desktop/tech-trends-by-linkedin-jobs/data/linkedin.db'
con = sqlite3.connect(settings.sqlite_path)
df = pd.read_sql_query("SELECT * FROM jobs", con=con)
df = df[df.keyword == 'Data Scientist']
df.head()

,row_id,created_at,modified_at,task_id,keyword,country,job_id,company,title,location,salary,description,skills_frameworks,skills_databases,skills_platform,skills_ds_tags,skills_prog_langs
0,1,2022-02-26 21:55:34,2022-02-27 01:10:04.496747,7199c8ae418744d790f58c144ad83def,Data Scientist,United States,2932249142,Starbucks,data scientist -Remote,"Seattle, WA",None,Now Brewing - Data Scientists! #tobeapartner\n...,,,Azure AWS,Data data forecasting visualization R PySpark,SQL Python Self Java Scala
1,2,2022-02-26 21:55:34,2022-02-27 01:10:05.472475,7199c8ae418744d790f58c144ad83def,Data Scientist,United States,2937320430,"Jamy Interactive, Inc",Data Scientist,United States,None,Job Description\nRESPONSIBILITIES\n· Deve...,pandas nltk,,,algorithms data visualization Statistics Mathe...,Python R
2,3,2022-02-26 21:55:34,2022-02-27 01:10:06.396433,7199c8ae418744d790f58c144ad83def,Data Scientist,United States,2936601561,Walmart,Data Scientist,"Dallas, TX",None,Position Summary... What You'll Do...\nWhat yo...,tensorflow torch,,,Data data Classification performance Statistic...,ML Processing Python Scala
3,4,2022-02-26 21:55:34,2022-02-27 01:10:07.471786,7199c8ae418744d790f58c144ad83def,Data Scientist,United States,2937385488,Evernote,"Data Scientist, Product Analytics",San Francisco Bay Area,None,About The Team\nThe Evernote Analytics team is...,,,,data visualization Performance Statistics Matl...,SQL Python R SAS B
4,5,2022-02-26 21:55:34,2022-02-27 01:10:08.311111,7199c8ae418744d790f58c144ad83def,Data Scientist,United States,2937325389,CVS Health,Data Scientist,"New York, NY",None,Job Description\nIt’s a new day in healthcare!...,,,,data Data performance algorithms,


# Visualization

In [29]:
def visualize(df, column, legend, keyword, save_to_img=True):
    df = df[column].dropna()
    vectorizer = TfidfVectorizer(analyzer='word')
    X = vectorizer.fit_transform(df)
    weights = np.asarray(X.mean(axis=0)).ravel().tolist()
    weights_df = pd.DataFrame({'value': vectorizer.get_feature_names(), 'weight': weights})
    weights_df.sort_values(by='weight', ascending=False).head()

    summary = weights_df.sort_values(by='weight', ascending=False)[:30]

    fig = px.bar(
        x=summary['value'],
        y=summary['weight'],
        color=summary['weight'],
        color_continuous_scale="Emrld",
    )
    fig.update_yaxes(title="Weight")
    fig.update_layout(
        showlegend=True,
        template="plotly_white",
        xaxis = go.layout.XAxis(tickangle = 45)
    )
    fig.show()
    if save_to_img:
        pio.write_image(fig, f'images/{str(legend).replace(" ", "")}_{str(keyword).replace(" ", "")}.png', width=None, height=None)

In [30]:
keywords = ['Data Scientist'] # df['keyword'].unique().tolist()
data = [
    {"column": "skills_prog_langs", "legend": "Programming Language"},
    {"column": "skills_platform", "legend": "Platforms"},
    {"column": "skills_databases", "legend": "Databases"},
    {"column": "skills_frameworks", "legend": "Frameworks"},
]

for keyword in keywords:
    tmp_df = df[df["keyword"] == keyword]
    for x in data:
        visualize(tmp_df, x['column'], x['legend'], f'Keyword: {keyword}')